In [1]:
library(tidyverse)
library(mediation)
library(brms)
library(dplyr)
library(lme4)
library(ggplot2)

── Attaching core tidyverse packages ─────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   4.0.0     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.1.0     
── Conflicts ───────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the ]8;;http://conflicted.r-lib.org/conflicted package]8;; to force all conflicts to become errors


Warning messages:
1: package ‘tidyverse’ was built under R version 4.3.3 
2: package ‘lubridate’ was built under R version 4.3.2 


Loading required package: MASS

Attaching package: ‘MASS’

The following object is masked from ‘package:dplyr’:

    select

Loading required package: Matrix

Attaching package: ‘Matrix’

The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack

Loading required package: mvtnorm
Loading required package: sandwich
mediation: Causal Mediation Analysis
Version: 4.5.1



Warning messages:
1: package ‘mvtnorm’ was built under R version 4.3.3 
2: package ‘sandwich’ was built under R version 4.3.3 


Loading required package: Rcpp
Loading 'brms' package (version 2.22.0). Useful instructions
can be found by typing help('brms'). A more detailed introduction
to the package is available through vignette('brms_overview').

Attaching package: ‘brms’

The following object is masked from ‘package:stats’:

    ar



Warning message:
package ‘brms’ was built under R version 4.3.3 



Attaching package: ‘lme4’

The following object is masked from ‘package:brms’:

    ngrps



# Import

In [3]:
df_long <- read.csv("df_RCT.csv")

# Sleep

In [4]:

mediators <- c("smip_pre_max", "smip_post_max", "pif_pre_max", "pif_post_max", "sindex_pre_max", "sindex_post_max", "anxiety_sum", "depression_sum", "sfpa_sum")
outcomes  <- c("psqi_sum", "psqi_disturbances")

# ---- storage for CSV ----
results_list <- list()

# ---- PDF diagnostics ----
pdf("mediation_diagnostics.pdf", width = 8, height = 6)

row_index <- 1

for (med in mediators) {
  for (out in outcomes) {
    
    message("Running: ", med, " -> ", out)

    # Subset complete data
    vars_needed <- c(med, out, "time_point", "record_id")
    temp_data <- df_long[complete.cases(df_long[vars_needed]), ]
    
    if (nrow(temp_data) < 10) {
      message("Skipping (too few rows).")
      next
    }
    
    # Build formulas
    med_formula <- as.formula(paste0(med, " ~ time_point + (1 | record_id)"))
    out_formula <- as.formula(paste0(out, " ~ time_point + ", med, " + (1 | record_id)"))
    
    # Fit models with error handling
    med.fit <- try(lmer(med_formula, data = temp_data, REML = FALSE), silent = TRUE)
    if (inherits(med.fit, "try-error")) next
    
    out.fit <- try(lmer(out_formula, data = temp_data, REML = FALSE), silent = TRUE)
    if (inherits(out.fit, "try-error")) next
    
    # Mediation analysis
    med.out <- try(
      mediate(
        model.m = med.fit,
        model.y = out.fit,
        treat = "time_point",
        mediator = med,
        sims = 2000
      ), 
      silent = TRUE
    )
    
    if (inherits(med.out, "try-error")) next
    
    #### ---- Save results for CSV ---- ####
    sum_out <- summary(med.out)
    
    results_list[[row_index]] <- data.frame(
      mediator = med,
      outcome  = out,
      ACME_est = sum_out$d0,
      ACME_p   = sum_out$d0.p,
      ADE_est  = sum_out$z0,
      ADE_p    = sum_out$z0.p,
      Total_est = sum_out$tau.coef,
      Total_p   = sum_out$tau.p,
      PropMed  = sum_out$n0,
      stringsAsFactors = FALSE
    )
    row_index <- row_index + 1
    
    #### ---- Diagnostics: Density plot of simulation draws ---- ####
    plot_df <- data.frame(
      ACME = med.out$d0.sims,
      ADE  = med.out$z0.sims
    )
    
    print(
      ggplot(plot_df, aes(x = ACME)) +
        geom_density() +
        ggtitle(paste("ACME Simulation Distribution:", med, "→", out))
    )
    
    print(
      ggplot(plot_df, aes(x = ADE)) +
        geom_density() +
        ggtitle(paste("ADE Simulation Distribution:", med, "→", out))
    )
    
  }
}

dev.off()

# ---- Write CSV ----
results_df <- bind_rows(results_list)
write.csv(results_df, "mediation_results.csv", row.names = FALSE)

message("Done. CSV and PDF created.")


Running: smip_pre_max -> psqi_sum


: Error in `geom_density()`:
! Problem while computing aesthetics.
ℹ Error occurred in the 1st layer.
Caused by error:
! object 'ACME' not found